# Cargue de librerias

In [99]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import re
from unicodedata import normalize
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.float_format = '{:,.2f}'.format
import pyxlsb
from pyxlsb import convert_date

# Cargue de Datos

In [100]:
base = pd.read_csv('Reporte_Delito_Violencia_Intrafamiliar_Polic_a_Nacional.csv')

C:\Users\enive\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Parámetros

In [101]:
#periodo de tiempo para filtrar la base
periodo = [2010,2020]

### Análisis descriptivo de los datos

In [102]:
#revisamos la estructura de la data
base.head()

,DEPARTAMENTO,MUNICIPIO,CODIGO DANE,ARMAS MEDIOS,FECHA HECHO,GENERO,GRUPO ETARIO,CANTIDAD
0,ATLÁNTICO,BARRANQUILLA (CT),8001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
1,BOYACÁ,DUITAMA,15238000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
2,CAQUETÁ,PUERTO RICO,18592000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
3,CASANARE,MANÍ,85139000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
4,CUNDINAMARCA,BOGOTÁ D.C. (CT),11001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1


In [103]:
#Variables identificadas
base.columns

Index(['DEPARTAMENTO', 'MUNICIPIO', 'CODIGO DANE', 'ARMAS MEDIOS',
       'FECHA HECHO', 'GENERO', 'GRUPO ETARIO', 'CANTIDAD'],
      dtype='object')

In [104]:
#conteo de datos pertidos (nulos)
base.isnull().sum()

DEPARTAMENTO       0
MUNICIPIO          0
CODIGO DANE        0
ARMAS MEDIOS       2
FECHA HECHO        0
GENERO             3
GRUPO ETARIO    1615
CANTIDAD           0
dtype: int64

In [105]:
#tipos de datos recibidos
base.dtypes

DEPARTAMENTO    object
MUNICIPIO       object
CODIGO DANE     object
ARMAS MEDIOS    object
FECHA HECHO     object
GENERO          object
GRUPO ETARIO    object
CANTIDAD         int64
dtype: object

In [106]:
#dimensiones de la base
base.shape

(476970, 8)

### Estandarización de los datos

In [107]:
#estandarización de la varible ARMAS MEDIOS
base['ARMAS MEDIOS'].unique()
base['ARMAS MEDIOS'] = np.where(base['ARMAS MEDIOS'] == 'CORTANTES', 'ARMA BLANCA / CORTOPUNZANTE',
                               np.where(base['ARMAS MEDIOS'] == 'CORTOPUNZANTES', 'ARMA BLANCA / CORTOPUNZANTE',
                               np.where(base['ARMAS MEDIOS'] == 'NO REPORTADO', 'NO REPORTA',
                               np.where(base['ARMAS MEDIOS'] == '-', 'NO REPORTA', base['ARMAS MEDIOS']))))
base['ARMAS MEDIOS'] = base['ARMAS MEDIOS'].fillna('NO REPORTA')

base['ARMAS MEDIOS'] = [re.sub(
                r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
                normalize( "NFD", x),  0, re.I) for x in base['ARMAS MEDIOS'] ]

array(['ARMA BLANCA / CORTOPUNZANTE', 'ARMA DE FUEGO', 'CONTUNDENTES',
       'CORTANTES', 'CORTOPUNZANTES', 'NO REPORTADO', 'PUNZANTES',
       'SIN EMPLEO DE ARMAS', 'NO REPORTA', 'ESCOPOLAMINA', '-', nan],
      dtype=object)

In [108]:
#estandarización de la variable GENERO
base['GENERO'].unique()
base['GENERO'] = np.where(base['GENERO'] == '-', 'NO REPORTA', base['GENERO'])
base['GENERO'] = base['GENERO'].fillna('NO REPORTA')
base['GENERO'] = [re.sub(
                r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
                normalize( "NFD", x),  0, re.I) for x in base['GENERO'] ]
base['GENERO'].unique()

array(['MASCULINO', 'FEMENINO', 'NO REPORTA', '-', nan], dtype=object)

array(['MASCULINO', 'FEMENINO', 'NO REPORTA'], dtype=object)

In [109]:
#estandarización de la variable GRUPO ETARIO
base['GRUPO ETARIO'].unique()
base['GRUPO ETARIO'] = base['GRUPO ETARIO'].fillna('NO REPORTA')

array(['ADULTOS', 'ADOLESCENTES', 'MENORES', 'NO REPORTA', nan],
      dtype=object)

In [110]:
#estandarización de la variable DEPARTAMENTO
base['DEPARTAMENTO'] = [re.sub(
                r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
                normalize( "NFD", x),  0, re.I) for x in base['DEPARTAMENTO'] ]

In [111]:
#estandarización de la variable CODIGO DANE
base['CODIGO DANE'] = [str(x)[:-3] for x in base['CODIGO DANE']]

In [112]:
#creación de la variable CAPITAL_TURISTICA
base['CAPITAL_TURISTICA'] = 0
base.loc[base.MUNICIPIO.str.contains('CT'), 'CAPITAL_TURISTICA']=1

In [113]:
#estandarización de la variable MUNICIPIO
base['MUNICIPIO'] = [re.sub(
                r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
                normalize( "NFD", x),  0, re.I) for x in base['MUNICIPIO']]
base['MUNICIPIO'] = base.MUNICIPIO.str.replace('CT', '') 
base['MUNICIPIO'] = base.MUNICIPIO.str.replace(')', '') 
base['MUNICIPIO'] = base.MUNICIPIO.str.replace('(', '') 

C:\Users\enive\AppData\Local\Temp/ipykernel_25096/1665858235.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base['MUNICIPIO'] = base.MUNICIPIO.str.replace(')', '')
C:\Users\enive\AppData\Local\Temp/ipykernel_25096/1665858235.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base['MUNICIPIO'] = base.MUNICIPIO.str.replace('(', '')


In [114]:
#estandarización de la variable FECHA HECHO
lista = []
for i in base['FECHA HECHO']:
    try:
        
        lista.append(convert_date(int(i)).strftime('%d/%m/%Y'))
    except:
        lista.append(i)
        
base['FECHA HECHO'] = lista

In [115]:
#creación de las variables año mes día
Fecha = base['FECHA HECHO'].str.split('/',expand = True)
Fecha.columns = ['DIA', 'MES', 'AÑO']
Fecha['DIA'] = Fecha['DIA'].astype(int)
Fecha['MES'] = Fecha['AÑO'].astype(int)
Fecha['AÑO'] = Fecha['AÑO'].astype(int)
base = pd.concat([base, Fecha], axis = 1)

In [116]:
#se filtra la base
base = base[(base.AÑO >= periodo[0]) & (base.AÑO <= periodo[1]) ]

In [119]:
#previsualización de la base antes de exportar
base.head()

,DEPARTAMENTO,MUNICIPIO,CODIGO DANE,ARMAS MEDIOS,FECHA HECHO,GENERO,GRUPO ETARIO,CANTIDAD,CAPITAL_TURISTICA,DIA,MES,AÑO
0,ATLANTICO,BARRANQUILLA,8001,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1,1,1,2010,2010
1,BOYACA,DUITAMA,15238,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1,0,1,2010,2010
2,CAQUETA,PUERTO RICO,18592,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1,0,1,2010,2010
3,CASANARE,MANI,85139,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1,0,1,2010,2010
4,CUNDINAMARCA,BOGOTA D.C.,11001,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1,1,1,2010,2010


# Exportar Datos

In [120]:
base.to_csv('base_estandarizada_visualizacion.csv', sep = ',', index=False)